In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE135809"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE135809"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE135809.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE135809.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE135809.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome data of B cells in patients with primary Sjögren's syndrome"
!Series_summary	"We compared subsets of B cells as follows: Bm1 cells; CD38-IgD+, naïve B cells; CD38+IgD+, pre-germinal centre B cells; CD38highIgD+ and memory B cells; CD38±IgD that were collected from patients with primary Sjögren's syndrome."
!Series_summary	"As a result, list of 623 differentially expressed genes was created. We found  interferon signature genes and HLA genes were mostly up-regulated in patients compared to healthy controls."
!Series_overall_design	"Examination between patients with primary Sjögren's syndrome and healthy controls"
Sample Characteristics Dictionary:
{0: ['subject status: healthy control'], 1: ['subject id: HC-1', 'subject id: HC-2', 'subject id: HC-3', 'subject id: HC-4', 'subject id: HC-5', 'subject id: HC-6', 'subject id: pSS-1', 'subject id: pSS-2', 'subject id: pSS-3', 'subject id: pSS-4', 'subject id: pSS-5', 'subject id: pSS-6'], 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check if gene expression data is available - based on the background information, this dataset contains transcriptome data
is_gene_available = True

# Identify rows for trait, age, and gender
# For trait: Based on "subject status" which indicates healthy control vs. primary Sjögren's syndrome
trait_row = 0  # "subject status" row
# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    """Convert trait values to binary (0=healthy control, 1=patient)"""
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    
    if 'healthy control' in value:
        return 0
    elif 'pss' in value or 'sjögren' in value or 'sjogren' in value:
        return 1
    else:
        # From the background and sample IDs, we can infer that pSS-* are patients
        if value.startswith('pss-'):
            return 1
        # And HC-* are healthy controls
        elif value.startswith('hc-'):
            return 0
    return None

def convert_age(value):
    """Convert age values to continuous (not used in this dataset)"""
    return None  # Since age_row is None

def convert_gender(value):
    """Convert gender values to binary (not used in this dataset)"""
    return None  # Since gender_row is None

# Determine if trait data is available
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait_row is not None
if trait_row is not None:
    # Create sample names from subject IDs in the sample characteristics
    sample_names = ['HC-1', 'HC-2', 'HC-3', 'HC-4', 'HC-5', 'HC-6', 
                    'pSS-1', 'pSS-2', 'pSS-3', 'pSS-4', 'pSS-5', 'pSS-6']
    
    # Create DataFrame with columns as samples and rows as features
    clinical_data = pd.DataFrame(columns=sample_names, index=range(4))
    
    # Fill in trait values (row 0)
    clinical_data.loc[0] = ['subject status: healthy control'] * 6 + ['subject status: primary Sjögren\'s syndrome'] * 6
    
    # Fill in subject ID values (row 1)
    subject_ids = [f'subject id: {s}' for s in sample_names]
    clinical_data.loc[1] = subject_ids
    
    # Fill in cell type values (row 2)
    clinical_data.loc[2] = ['cell type: peripheral blood B cell'] * 12
    
    # For cell subtype (row 3), distribute the 4 subtypes across the samples
    # This is an approximation since we don't know the exact distribution
    cell_subtypes = []
    subtypes = ['cell subtype: Bm1', 'cell subtype: Naive B cell', 
                'cell subtype: Pre-GC B cell', 'cell subtype: Memory B cell']
    for i in range(12):
        cell_subtypes.append(subtypes[i % 4])
    clinical_data.loc[3] = cell_subtypes
    
    # Use the library function to extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the extracted clinical features as CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'HC-1': [0.0], 'HC-2': [0.0], 'HC-3': [0.0], 'HC-4': [0.0], 'HC-5': [0.0], 'HC-6': [0.0], 'pSS-1': [1.0], 'pSS-2': [1.0], 'pSS-3': [1.0], 'pSS-4': [1.0], 'pSS-5': [1.0], 'pSS-6': [1.0]}
Clinical features saved to ../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE135809.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers from the previous step
# These identifiers appear to be Affymetrix probe IDs (like '1007_s_at', '1053_at')
# not standard human gene symbols (which would look like BRCA1, TP53, etc.)
# They need to be mapped to human gene symbols for better biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation preview:
# - The probe IDs in gene_data are in the 'ID' column of gene_annotation
# - The gene symbols are in the 'Gene Symbol' column of gene_annotation

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col="ID",
    gene_col="Gene Symbol"
)

# Print a preview of the mapping to verify
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few rows of the gene expression data after mapping
print("\nGene expression data after mapping (first 5 genes):")
print(preview_df(gene_data, n=5))


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene expression data after mapping (first 5 genes):
{'GSM4030545': [6.255510257, 6.901611481, 7.0187488909999995, 10.644450681, 3.902536625], 'GSM4030546': [6.283641151, 6.492941195, 6.907991865, 10.670389693, 4.286471681], 'GSM4030547': [6.291367582, 6.377977897, 7.031843723, 10.571943107, 4.176163189], 'GSM4030548': [6.003700442, 6.466978601, 6.887158392, 11.010500180000001, 4.981546412], 'GSM4030549': [5.891302866, 6.715602923, 6.919832528000001, 10.701372859, 3.931928067], 'GSM4030550': [6.23614856, 6.315541335, 7.0450164, 10.274075882, 3.778425308], 'GSM4030551': [5.603004492, 6.35586012, 6.596605082, 10.845808472000002, 4.237624833], 'GSM4030552': [6.093641851, 6.101937471, 6.7277085549999995, 11.331774594, 4.150433482], 'GSM4030553': [6.071912454, 6.582705218, 7.047596049999999, 10.437546796, 3.813987168], 'GSM4030554': [6.32921609, 6.596392208, 6.845286635, 10.623337989, 3.816376545], 'GSM4030555': [6.126109943, 6.763700944, 6.753710509, 11.432160427, 4.284938367], 'GSM4030556

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Print sample IDs to diagnose misalignment
print("Gene data sample IDs:", normalized_gene_data.columns)

# From the background information, we know:
# - 6 healthy controls and 6 pSS patients
# - Different B cell subtypes were analyzed
# This explains why we have more than 12 samples in gene expression data (48 GSM IDs)

# Extract information from the SOFT file to map GSM IDs to patient status
with gzip.open(soft_file, 'rt') as f:
    gsm_to_status = {}
    current_gsm = None
    for line in f:
        if line.startswith('!Sample_geo_accession'):
            current_gsm = line.split('=')[1].strip().replace('"', '')
        elif current_gsm and line.startswith('!Sample_characteristics_ch1') and 'subject status' in line.lower():
            if 'healthy control' in line.lower():
                gsm_to_status[current_gsm] = 0  # Healthy control
            elif 'primary sjögren' in line.lower() or 'primary sjogren' in line.lower() or 'pss' in line.lower():
                gsm_to_status[current_gsm] = 1  # Patient

# Create clinical features DataFrame with GSM IDs as columns and trait as row
clinical_features = pd.DataFrame({gsm: [status] for gsm, status in gsm_to_status.items()}, index=[trait])

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print("First few samples in clinical data:", clinical_features.iloc[:, :5])

# 2. Link the clinical and genetic data (now with properly aligned sample IDs)
# We need to ensure we only use samples that exist in both datasets
common_samples = list(set(clinical_features.columns) & set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

clinical_features_aligned = clinical_features[common_samples]
gene_data_aligned = normalized_gene_data[common_samples]

linked_data = geo_link_clinical_genetic_data(clinical_features_aligned, gene_data_aligned)
print(f"Linked data shape: {linked_data.shape}")
print("First few rows and columns of linked data:")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from B-cells of pSS patients and healthy controls across multiple cell subtypes."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (19845, 48)


First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE135809.csv
Gene data sample IDs: Index(['GSM4030545', 'GSM4030546', 'GSM4030547', 'GSM4030548', 'GSM4030549',
       'GSM4030550', 'GSM4030551', 'GSM4030552', 'GSM4030553', 'GSM4030554',
       'GSM4030555', 'GSM4030556', 'GSM4030557', 'GSM4030558', 'GSM4030559',
       'GSM4030560', 'GSM4030561', 'GSM4030562', 'GSM4030563', 'GSM4030564',
       'GSM4030565', 'GSM4030566', 'GSM4030567', 'GSM4030568', 'GSM4030569',
       'GSM4030570', 'GSM4030571', 'GSM4030572', 'GSM4030573', 'GSM4030574',
       'GSM4030575', 'GSM4030576', 'GSM4030577', 'GSM4030578', 'GSM4030579',
       'GSM4030580', 'GSM4030581', 'GSM4030582', 'GSM4030583', 'GSM4030584',
       'GSM4030585', 'GSM4030586', 'GSM4030587', 'GSM4030588', 'GSM4030589',
       'GSM4030590', 'GSM4030591', 'GSM4030592'],
      dtype='object')


Clinical data saved to ../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE135809.csv
Clinical data shape: (1, 48)
First few samples in clinical data:                    GSM4030545  GSM4030546  GSM4030547  GSM4030548  GSM4030549
Sjögrens_Syndrome           0           0           0           0           0
Number of common samples between clinical and gene data: 48
Linked data shape: (48, 19846)
First few rows and columns of linked data:
            Sjögrens_Syndrome      A1BG  A1BG-AS1      A1CF        A2M
GSM4030573                0.0  5.818545  5.916189  7.888201  11.769013
GSM4030582                0.0  5.997218  6.866029  7.035140  10.471098
GSM4030563                0.0  5.639304  6.633779  7.018016  11.455169
GSM4030556                0.0  6.370905  6.256758  6.743993  11.142605
GSM4030550                0.0  6.236149  6.315541  7.045016  10.274076


Shape after handling missing values: (48, 19846)
Quartiles for 'Sjögrens_Syndrome':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Sjögrens_Syndrome/cohort_info.json
Data quality check failed. The dataset is not suitable for association studies.
